# Metric Exploration

Here we process and plot metrics for the UST depeg.

In [74]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [75]:
import os
import sys
import requests as req
import json

from datetime import datetime, timedelta

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

new_path = os.getcwd().replace('/notebooks', '')
print(f'Adding {new_path} to sys.path')

if new_path not in sys.path:
    sys.path.append(new_path)

# Local imports
from curvemetrics.src.classes.datahandler import DataHandler
from curvemetrics.src.classes.datafetcher import DataFetcher
from curvemetrics.src.classes.metricsprocessor import MetricsProcessor

Adding /root/curve-lp-metrics to sys.path


In [76]:
plt.rcParams["font.family"] = "serif"
plt.rcParams.update({'font.size': 10})

In [78]:
datahandler = DataHandler()
token_metadata = datahandler.get_token_metadata()
pool_metadata = datahandler.get_pool_metadata()

pool = "0xceaf7747579696a2f0bb206a14210e3c9e6fb269" # UST

start_ts, start_block = DataFetcher.get_block(datetime.fromisoformat("2022-04-01"))
end_ts, end_block = DataFetcher.get_block(datetime.fromisoformat("2022-06-01"))

In [79]:
swaps_data = datahandler.get_swaps_data(pool, start_ts, end_ts)
lp_data = datahandler.get_lp_data(pool, start_ts, end_ts)
pool_data = datahandler.get_pool_data(pool, start_ts, end_ts)

In [80]:
metricsprocessor = MetricsProcessor(pool_metadata, token_metadata)
pool_metrics = metricsprocessor.process_metrics_for_pool(pool, pool_data, swaps_data, lp_data)
pool_metrics.head()

,giniCoefficient,shannonsEntropy,USDC.netSwapFlow,USDC.absSwapFlow,USDC.PIN,UST.netSwapFlow,UST.absSwapFlow,UST.PIN,USDT.netSwapFlow,USDT.absSwapFlow,...,DAI.netSwapFlow,DAI.absSwapFlow,DAI.PIN,3Crv.netSwapFlow,3Crv.absSwapFlow,3Crv.PIN,UST.netLPFlow,UST.absLPFlow,3Crv.netLPFlow,3Crv.absLPFlow
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-04-01 00:00:00,0.5,8.374852e-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
2022-04-01 00:01:00,0.5,8.374852e-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
2022-04-01 00:02:00,0.5,8.374852e-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
2022-04-01 00:03:00,0.5,8.374852e-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
2022-04-01 00:04:00,0.5,8.374852e-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


In [81]:
tokens = {token_metadata[v]['symbol']:v for v in set(swaps_data['tokenBought'])}
tokens

{'USDC': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48',
 'UST': '0xa693b19d2931d498c5b318df961919bb4aee87a5',
 'USDT': '0xdac17f958d2ee523a2206206994597c13d831ec7',
 'DAI': '0x6b175474e89094c44da98b954eedeac495271d0f',
 '3Crv': '0x6c3f90f043a72fa612cbac8115ee7e52bde6e490'}

In [ ]:
ohlcvs = {}
for k, v in tokens.items():
    if k == "3Crv":
        continue
    ohlcv = datahandler.get_ohlcv_data(v, start=start_ts, end=end_ts)
    ohlcvs[k] = ohlcv